# 1. Installing

In [6]:
!pip install transformers

In [7]:
!pip install --upgrade pip

In [1]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

# 2. Load Summarization Pipeline

In [2]:
summarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


# 3. Get Blog post

In [5]:
URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"

In [6]:
r = requests.get(URL)

In [7]:
r.text

AttributeError: can't set attribute 'text'

# 4. Chunk Text

In [17]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

You can directly change ARTICLE in here for which paragraph you want.After that, you can jump to line 24. Warnning, accuracy can be change. 

In [18]:
ARTICLE = """
ROUGE, or Recall-Oriented Understudy for Gisting Evaluation,[1] is a set of metrics and a software package used for evaluating automatic summarization and machine translation software in natural language processing. The metrics compare an automatically produced summary or translation against a reference or a set of references (human-produced) summary or translation.
"""

In [19]:
max_chunk = 500

Adjusting html version of paragraph

In [20]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [21]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [22]:
len(chunks)


1

In [23]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

Your max_length is set to 120, but your input_length is only 76. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)


In [24]:
res[0]

{'summary_text': ' Recall-Oriented Understudy for Gisting Evaluation is a set of metrics and a software package used for evaluating automatic summarization and machine translation software in natural language processing . The metrics compare an automatically produced summary or translation against a reference .'}

In [25]:
' '.join([summ['summary_text'] for summ in res])

' Recall-Oriented Understudy for Gisting Evaluation is a set of metrics and a software package used for evaluating automatic summarization and machine translation software in natural language processing . The metrics compare an automatically produced summary or translation against a reference .'

In [26]:
text = ' '.join([summ['summary_text'] for summ in res])

In [27]:
text

' Recall-Oriented Understudy for Gisting Evaluation is a set of metrics and a software package used for evaluating automatic summarization and machine translation software in natural language processing . The metrics compare an automatically produced summary or translation against a reference .'